In [108]:
import numpy as np
import pandas as pd
import re

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.layers import Flatten
from keras.utils.np_utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [109]:
data = pd.read_csv('/Users/niticon/ling575/ted-talks/xls_ted_main.csv')
data = data[['gender', 'ratings']]

data.head(1000)

,gender,ratings
0,m,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {..."
1,m,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i..."
2,m,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i..."
3,f,"[{'id': 3, 'name': 'Courageous', 'count': 760}..."
4,m,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}..."
...,...,...
995,f,"[{'id': 1, 'name': 'Beautiful', 'count': 298},..."
996,m,"[{'id': 24, 'name': 'Persuasive', 'count': 67}..."
997,m,"[{'id': 24, 'name': 'Persuasive', 'count': 106..."
998,m,"[{'id': 10, 'name': 'Inspiring', 'count': 1729..."


In [175]:
data['ratings'] = data['ratings'].apply(lambda x: x.lower())
data['ratings'] = data['ratings'].apply((lambda x: re.sub(r'[^a-zA-z0-9\s]','',x)))
data['ratings'] = data['ratings'].apply((lambda x: re.sub('id','',x)))
data['ratings'] = data['ratings'].apply((lambda x: re.sub('name','',x)))
data['ratings'] = data['ratings'].apply((lambda x: re.sub('count','',x)))
data['ratings'] = data['ratings'].apply((lambda x: re.sub(r'[0-9]','',x)))

for idx,row in data.iterrows():
    row[0] = row[0].replace('id', r'[0-9]')

max_features = 5000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['ratings'].values)

X = tokenizer.texts_to_sequences(data['ratings'].values)
X = pad_sequences(X)

In [176]:
data['ratings'].head()

0    [   funny      beautiful      ingenious      c...
1    [   funny      courageous      confusing      ...
2    [   funny      courageous      ingenious      ...
3    [   courageous      beautiful      confusing  ...
4    [   ingenious      funny      beautiful      c...
Name: ratings, dtype: object

In [189]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, 64, input_length=X.shape[1]))
model.add(Flatten())
model.add(Dense(300, activation='relu'))
model.add(Dense(2460, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'],)

print(model.summary())

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 14, 64)            320000    
_________________________________________________________________
flatten_19 (Flatten)         (None, 896)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 300)               269100    
_________________________________________________________________
dense_39 (Dense)             (None, 2460)              740460    
Total params: 1,329,560
Trainable params: 1,329,560
Non-trainable params: 0
_________________________________________________________________
None


In [190]:
Y = pd.get_dummies(data['ratings']).values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state = 42)

print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1648, 14) (1648, 2460)
(812, 14) (812, 2460)


In [191]:
batch_size = 16
model.fit(X_train, Y_train, epochs=32, batch_size=batch_size, verbose = 1)

Epoch 1/32
103/103 [==============================] - 0s 4ms/step - loss: 0.1276 - accuracy: 0.0000e+00
Epoch 2/32
103/103 [==============================] - 0s 5ms/step - loss: 0.0040 - accuracy: 0.0000e+00
Epoch 3/32
103/103 [==============================] - 0s 5ms/step - loss: 0.0037 - accuracy: 0.0000e+00
Epoch 4/32
103/103 [==============================] - 1s 5ms/step - loss: 0.0036 - accuracy: 0.0000e+00
Epoch 5/32
103/103 [==============================] - 0s 5ms/step - loss: 0.0036 - accuracy: 0.0000e+00
Epoch 6/32
103/103 [==============================] - 0s 4ms/step - loss: 0.0037 - accuracy: 0.0000e+00
Epoch 7/32
103/103 [==============================] - 0s 4ms/step - loss: 0.0037 - accuracy: 0.0000e+00
Epoch 8/32
103/103 [==============================] - 0s 4ms/step - loss: 0.0037 - accuracy: 0.0000e+00
Epoch 9/32
103/103 [==============================] - 0s 4ms/step - loss: 0.0037 - accuracy: 0.0000e+00
Epoch 10/32
103/103 [==============================] - 0s 4ms/st

In [192]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]

X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]

#score, acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
score, acc = model.evaluate(X_validate, Y_validate, verbose = 1)

print("score: %.2f" % score)
print("acc: %.2f" % acc)

26/26 [==============================] - 0s 3ms/step - loss: 0.0065 - accuracy: 0.0000e+00
score: 0.01
acc: 0.00
